In [1]:
import pygame
import moderngl
import numpy as np
import glm
from LoadObject import getObjectData

pygame 2.6.0 (SDL 2.28.4, Python 3.12.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
# Initialize Pygame
pygame.init()

# Set OpenGL version and create an OpenGL context
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_PROFILE_MASK, pygame.GL_CONTEXT_PROFILE_CORE)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MAJOR_VERSION, 4)
pygame.display.gl_set_attribute(pygame.GL_CONTEXT_MINOR_VERSION, 1)
pygame.display.set_mode((800, 600), pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE)

# Get the ModernGL context
gl = moderngl.create_context()
gl.enable(moderngl.DEPTH_TEST)

# Dictionary mapping keys 1-6 to corresponding OBJ files
obj_files = {
    pygame.K_1: "Teapot.obj",
    pygame.K_2: "Tetrahedron.obj",
    pygame.K_3: "Hexahedron.obj",
    pygame.K_4: "Octahedron.obj",
    pygame.K_5: "Dodecahedron.obj",
    pygame.K_6: "Icosahedron.obj"
}

# Initialize window size variables
window_width, window_height = 800, 600

2024-10-01 18:08:53.794 Python[97975:5261231] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.


In [4]:
## Vertex Shader
vertex_shader_code = '''
#version 330 core

layout (location = 0) in vec3 in_position;
layout (location = 1) in vec3 in_normal;

uniform mat4 view;
uniform mat4 perspective;

out vec3 f_normal;

void main() {
    f_normal = in_normal;
    gl_Position = perspective * view * vec4(in_position, 1.0);
}
'''

In [5]:
# Fragment Shader
fragment_shader_code = '''
#version 330 core

in vec3 f_normal;
out vec4 out_color;

void main() {
    out_color = vec4(0.5 * (normalize(f_normal) + 1.0), 1.0);
}
'''

In [6]:

# Compile shaders and link them into a program
shaderProgram = gl.program(
    vertex_shader=vertex_shader_code,
    fragment_shader=fragment_shader_code
)

In [7]:
# Load all objects at the start and store them
loaded_objects = {}
for key, obj_file in obj_files.items():
    renderable, modelMatrix, bounds = load_object(obj_file)
    loaded_objects[key] = (renderable, modelMatrix, bounds)

Normal exists
Normal computed.
Normal computed.
Normal computed.
Normal computed.
Normal computed.


In [8]:
## Set the initial object (Teapot.obj)
current_key = pygame.K_1  # Start with key 1 (Teapot.obj)
renderable, modelMatrix, bounds = loaded_objects[current_key]

# Pygame window setup
pygame.display.set_caption("Assignment 5: Nelson Herrera")

# Main loop
running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT or (
            event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE
        ):
            running = False
        
        # Check if a number key between 1-6 is pressed
        if event.type == pygame.KEYDOWN and event.key in obj_files:
            current_key = event.key
            renderable, modelMatrix, bounds = loaded_objects[current_key]
        
        # Handle window resize event
        if event.type == pygame.VIDEORESIZE:
            window_width, window_height = event.w, event.h
            pygame.display.set_mode(
                (window_width, window_height), 
                pygame.OPENGL | pygame.DOUBLEBUF | pygame.RESIZABLE
            )
            gl.viewport = (0, 0, window_width, window_height)

    # Clear the screen
    gl.clear(0.2, 0.2, 0.2)
    
    # Compute aspect ratio
    aspect_ratio = window_width / window_height

    # Compute the time-based rotation angle
    angle_in_radian = pygame.time.get_ticks() / 1000.0

    # Compute the starting vector (1,1,0) and normalize it
    starting_vector = glm.vec3(1, 1, 0)
    v = glm.normalize(starting_vector)  # This will be (1/sqrt(2), 1/sqrt(2), 0)

    # Rotate v around the Y-axis by the angle_in_radian
    rotation_matrix = glm.rotate(glm.mat4(1), angle_in_radian, glm.vec3(0, 1, 0))
    rotated_v = glm.vec3(rotation_matrix * glm.vec4(v, 0.0))

    # Compute the eye position
    eye_position = glm.vec3(bounds.center) + 2 * bounds.radius * rotated_v

    # Compute the up vector
    up_vector = glm.vec3(0, 1, 0)

    # Compute the view matrix
    view_matrix = glm.lookAt(eye_position, glm.vec3(bounds.center), up_vector)

    # Combine the view matrix with the model matrix
    view = view_matrix * modelMatrix  # Note: Multiplication order is important

    # Compute the perspective projection matrix
    near = 0.1
    far = 100
    fov = 60  # Field of view in degrees
    perspective_matrix = glm.perspective(glm.radians(fov), aspect_ratio, near, far)

    # Upload the matrices to the shader
    shaderProgram['view'].write(view)
    shaderProgram['perspective'].write(perspective_matrix)

    # Render the object
    renderable.render(moderngl.TRIANGLES)

    # Swap buffers
    pygame.display.flip()

# Cleanup
pygame.quit()
quit()